In [165]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
import os
import re
import sys
from joblib import Parallel
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options


In [166]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [167]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [168]:
def get_html_from_url(url):
  chrome_options = Options()
  chrome_options.add_argument("--headless")
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options) 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        flag = 0
  html = browser.page_source
  return html

In [169]:
def convert_time_to_year(time_list):
    today = date.today()
    years_updated = [] 
    updated='UNKNOWN'
    
    for time in time_list:
        time = time.replace(' ago', '')

        time_word = re.sub(r'\d+', '', time)
        time_word.replace(' ', '')

        if 'days' in time_word.lower() or 'hours' in time_word.lower():
            updated = today.year

        if 'months' in time_word.lower():
            time = time.replace(time_word, '')
            try:
                updated = (today - relativedelta(months=int(time))).year
            except:
                print("error in month date")
        
        if 'year' in time_word.lower():
            if 'years' in time_word.lower():
                time_word = time.replace('years','')
            else:
                 time_word = time.replace('year','')
            try:
                updated = today.year - int(time_word)
            except:
                print("error in year date")
        
        years_updated.append(updated)
    return years_updated

In [170]:
def row_index_with_low_upvotes(l):
    to_remove = []
    for i,element in enumerate(l):
        if 'k' in element.lower() or 'm' in element.lower():
            continue
        else:
            if element.lower() == 'vote'or int(element)<500:
                to_remove.append(i)
    return to_remove

In [171]:
#url_ex = "https://www.reddit.com/r/Futurology/"
#url_ex = "https://www.reddit.com/r/technology/"
url_ex = "https://www.reddit.com/r/tech/"
page_html = get_html_from_url(url_ex)

C:\Users\Ravit\AppData\Local\Temp\ipykernel_13712\3039072896.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options)


In [172]:
#go to filter by flair and extract domain pages
soup = BeautifulSoup(page_html)

In [173]:
big_box = soup.find("div", {"class":"rpBJOHq2PR60pnwJlUyP0"})

In [174]:
boxes = soup.find_all("div", {"data-testid":"post-container"})

In [175]:
posts = [box.find('h3', {"class":"_eYtD2XCVieq6emjKBH3m"}).find_next(text=True).strip() for box in boxes]

In [176]:
upvotes  = [box.find('div', {"class":"_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ"}).find_next(text=True).strip() for box in boxes]

In [177]:
domains = []
for box in boxes:
    domain = box.find('span', {
        'class':"_1jNPl3YUk6zbpLWdjaJT1r _2VqfzH0dZ9dIl3XWNxs42y aJrgrewN9C8x1Fusdx4hh _1Dl-kvSxyJMWO9nuoTof8N"
        })
    if domain is None:
        domains.append(np.nan)
    
    else:
        domains.append(domain.find_next(text=True).strip())
 

In [178]:
time_updated = [box.find('span', {'class':"_2VF2J19pUIMSLJFky-7PEI"}).find_next(text=True).strip() for box in boxes]

In [179]:
len(time_updated)

441

In [180]:
time_updated = convert_time_to_year(time_updated)

In [181]:
len(time_updated)

441

In [182]:
len(posts)

441

In [183]:
len(upvotes)

441

In [184]:
links_to_discussions = [] 
for i, box in enumerate(boxes):
    add_link = 'UNKNOWN'
    try:
        add_link = box.find('a', {'class':"_13svhQIUZqD9PVzFcLwOKT styled-outbound-link"})['href']
    except:
        print('No link found in box', i)
    links_to_discussions.append(add_link)

No link found in box 367
No link found in box 386
No link found in box 393
No link found in box 418
No link found in box 420
No link found in box 440


In [185]:
links_to_discussions

['https://www.businessinsider.in/tech/news/as-elon-musk-finalized-his-twitter-deal-company-founder-jack-dorsey-launched-a-beta-for-his-new-social-media-company/articleshow/95188700.cms',
 'https://www.newscientist.com/article/2344551-first-3d-quantum-accelerometer-could-let-ships-navigate-without-gps/',
 'https://techcrunch.com/2022/10/28/nasas-psyche-mission-to-a-metal-rich-asteroid-is-back-on-the-books-for-october-2023/',
 'https://arstechnica.com/gadgets/2022/09/a-history-of-arm-part-1-building-the-first-chip',
 'https://apnews.com/article/technology-european-union-business-christopher-wray-wally-adeyemo-aff98eba1c7470f9b0128c882971547d',
 'https://bullet.so/',
 'https://www.androidauthority.com/google-tensor-pixel-benchmarks-3225413/',
 'https://www.insiderintelligence.com/insights/drone-delivery-services/',
 'https://www.theregister.com/2022/10/28/microsoft_nadella_2022_pay_karma/',
 'https://petapixel.com/how-steve-sasson-invented-the-digital-camera/',
 'https://www.theverge.com/

In [186]:
df = pd.DataFrame([])
df['domains'] = pd.DataFrame(domains)
df['posts'] = pd.DataFrame(posts)
df['upvotes'] = pd.DataFrame(upvotes)
df['last_updated'] = pd.DataFrame(time_updated)
df['discussion_links'] = pd.DataFrame(links_to_discussions)


In [187]:
df

,domains,posts,upvotes,last_updated,discussion_links
0,NaN,"As Elon Musk finalized his Twitter deal, Jack ...",1.5k,2022,https://www.businessinsider.in/tech/news/as-el...
1,NaN,First 3D quantum accelerometer could let ships...,1.0k,2022,https://www.newscientist.com/article/2344551-f...
2,NaN,NASA’s Psyche mission to a metal-rich asteroid...,1.3k,2022,https://techcrunch.com/2022/10/28/nasas-psyche...
3,NaN,"A history of ARM, part 1: Building the first chip",52,2022,https://arstechnica.com/gadgets/2022/09/a-hist...
4,NaN,White House invites dozens of nations for rans...,Vote,2022,https://apnews.com/article/technology-european...
...,...,...,...,...,...
436,NaN,Millions Of iPhone Owners Wait For Batterygate...,8,2022,https://www.forbes.com/sites/barrycollins/2022...
437,NaN,Non-Hormonal Birth Control Pill for Men Could ...,3.1k,2022,https://gizmodo.com/a-birth-control-pill-for-m...
438,NaN,Hackers Drain Wedding Cash From Couples’ Zola ...,120,2022,https://www.vice.com/en/article/k7wmm9/hackers...
439,NaN,Woolly Mammoths at World’s Top 50 Innovators 2022,4,2022,https://www.codex.com/woolly-mammoths-at-world...


In [188]:
#remove posts with very low upvotes
idx_low_upvotes =row_index_with_low_upvotes(upvotes)

In [189]:
df.drop(idx_low_upvotes, axis=0, inplace=True)

In [190]:
df

,domains,posts,upvotes,last_updated,discussion_links
0,NaN,"As Elon Musk finalized his Twitter deal, Jack ...",1.5k,2022,https://www.businessinsider.in/tech/news/as-el...
1,NaN,First 3D quantum accelerometer could let ships...,1.0k,2022,https://www.newscientist.com/article/2344551-f...
2,NaN,NASA’s Psyche mission to a metal-rich asteroid...,1.3k,2022,https://techcrunch.com/2022/10/28/nasas-psyche...
9,NaN,How Steve Sasson Invented the Digital Camera,640,2022,https://petapixel.com/how-steve-sasson-invente...
11,NaN,"A new, genetically modified purple tomato may ...",4.3k,2022,https://edition.cnn.com/2022/09/17/business-fo...
...,...,...,...,...,...
421,NaN,Scientists may have found an affordable way to...,3.1k,2022,https://www.engadget.com/scientists-find-affor...
423,NaN,Tesla driver found the perfect place to keep h...,943,2022,https://finance.yahoo.com/news/tesla-driver-fo...
428,NaN,Massive asteroid crater left by dinosaur kille...,682,2022,https://nypost.com/2022/08/18/massive-asteroid...
434,NaN,ISS spacewalk interrupted by suit malfunction,2.7k,2022,https://www.engadget.com/iss-spacewalk-russia-...


In [191]:
save_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\reddit_RCS\31OCT22_tech_group_reddit.csv"
df.to_csv(save_path, index=False)
